In [4]:
import pdfplumber

# ✅ PDF 파일 경로
pdf_path = "../../data/pdf/강관비계 안전작업지침.pdf"

# ✅ PDF에서 줄 단위로 텍스트와 폰트 크기, 페이지 번호를 추출하는 함수
def extract_lines_with_font_and_page(pdf_path):
    extracted_data = []  # 최종 결과를 저장할 리스트

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):  # 페이지 번호 추가
            words = page.extract_words(extra_attrs=["fontname", "size", "top"])  # 폰트 크기, 위치 정보 포함하여 단어 추출

            lines = []  # 줄을 저장할 리스트
            current_line = []
            current_font_size = None
            current_line_top = None

            for word in words:
                font_size = word["size"]
                text = word["text"]
                line_top = word["top"]  # 줄의 위치 정보

                # 첫 번째 단어인 경우 초기화
                if current_line_top is None:
                    current_line_top = line_top
                    current_font_size = font_size

                # 줄 바꿈 감지 (top 값이 일정 이상 차이나면 새로운 줄로 간주)
                if abs(line_top - current_line_top) > 5:  # 줄 간격이 5pt 이상이면 줄 바꿈
                    lines.append((" ".join(current_line), round(current_font_size, 1), page_num))
                    current_line = []
                    current_font_size = font_size
                    current_line_top = line_top  # 새 줄의 top 값 업데이트

                current_line.append(text)

            # 마지막 줄 저장
            if current_line:
                lines.append((" ".join(current_line), round(current_font_size, 1), page_num))

            extracted_data.extend(lines)

    return extracted_data

# ✅ 함수 실행하여 결과 저장
text_with_fonts = extract_lines_with_font_and_page(pdf_path)

# ✅ 결과 샘플 출력
for line in text_with_fonts[:100]:
    print(line)


('KOSHA GUIDE', 12.0, 1)
('C - 30 - 2020', 12.0, 1)
('강관비계 안전작업 지침', 19.9, 1)
('2020. 12.', 19.9, 1)
('한국산업안전보건공단', 19.9, 1)
('안전보건기술지침의 개요', 12.0, 2)
('◦ 작성자 : 한국산업안전보건공단 송효근', 12.0, 2)
('◦ 개정자 : (사)한국건설안전협회 최순주', 12.0, 2)
('◦ 제․개정 경과', 12.0, 2)
('- 1998년 05월 건설안전분야 기준제정위원회 심의', 12.0, 2)
('- 1998년 09월 총괄기준제정위원회 심의', 12.0, 2)
('- 2006년 11월 건설안전분야 제정위원회 심의', 12.0, 2)
('- 2006년 12월 총괄제정위원회 심의', 12.0, 2)
('- 2011년 12월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)', 12.0, 2)
('- 2018년 6월 건설안전분야 제정위원회 심의(개정, 법규개정조항 반영)', 12.0, 2)
('- 2020년 11월 건설안전분야 표준제정위원회 심의(개정, 법규개정조항반영)', 12.0, 2)
('◦ 관련규격 및 자료', 12.0, 2)
('- 일본건설업 노동재해 방지협회 : 비계조립 작업안전', 12.0, 2)
('- 가설공사 표준시방서(2016년)', 12.0, 2)
('- 한국산업표준', 12.0, 2)
('- KOSHA GUIDE C-11-2012, 가설계단의 설치 및 사용 안전보건작업 지침 등', 12.0, 2)
('○ 관련 법규․규칙․고시 등', 12.0, 2)
('-「산업안전보건기준에 관한 규칙」제1편(총칙)', 12.0, 2)
('-「유해·위험작업의 취업 제한에 관한 규칙」[별표1]', 12.0, 2)
('- 고용노동부고시 제2020-3호(가설공사 표준안전 작업지침)', 12.0, 2)
('- 고용노동부고시 제2020-34호(방호장치 자율안전기준고시)', 12.0, 2)
('- 고용노동부고시 제2020-33호(방호장치 안전인증 고

In [5]:
# ✅ 폰트별 라인의 빈도수 계산
import pandas as pd

# text_with_fonts 리스트에서 폰트 크기만 추출하여 DataFrame 생성
font_size_counts = pd.DataFrame(text_with_fonts, columns=["text", "font_size", "page"])

# 폰트 크기별 빈도수 계산
font_size_stats = font_size_counts["font_size"].value_counts().reset_index()
font_size_stats.columns = ["font_size", "count"]

font_size_stats.describe

<bound method NDFrame.describe of    font_size  count
0       12.0    295
1       10.1     36
2       13.0     26
3       19.9      3
4        7.9      2
5       17.0      1>

### todo
1. 통계값 확인하여 소제목 폰트 크기와 내용 폰트 크기 추론
2. 1,2페이지 제거
2. 그림 및 표 제거
3. 특수문자 제거 "(/d)" 등
4. 한글을 포함하지 않는 라인 제거

In [6]:
import pdfplumber
import re
import pandas as pd

# 1. 텍스트 클리닝 함수
def clean_text(text):
    # (숫자) 형태의 특수문자 제거
    text = re.sub(r"\(\d+\)", "", text)
    # 한글, 영문, 숫자, 공백, 기본 문장부호(.,)를 제외한 문자 제거
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.,]", "", text)
    return text.strip()

# 2. PDF에서 라인별로 (텍스트, 폰트크기, 페이지) 추출
def extract_lines_from_pdf(pdf_path):
    extracted_lines = []  # (텍스트, 폰트크기, 페이지)를 저장할 리스트
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            # 1,2페이지는 건너뜁니다.
            if page_num in [1, 2]:
                continue
            
            # 폰트 크기, 폰트명, top(위치) 정보를 포함하여 단어 추출
            words = page.extract_words(extra_attrs=["fontname", "size", "top"])
            current_line_words = []
            current_line_top = None
            current_font_size = None
            
            for word in words:
                text = word["text"]
                font_size = word["size"]
                top = word["top"]
                
                # 첫 단어이면 초기화
                if current_line_top is None:
                    current_line_top = top
                    current_font_size = font_size
                
                # top 차이가 5pt 이상이면 새로운 줄로 판단
                if abs(top - current_line_top) > 5:
                    line_text = " ".join(current_line_words)
                    extracted_lines.append((line_text, round(current_font_size, 1), page_num))
                    current_line_words = []
                    current_line_top = top
                    current_font_size = font_size
                    
                current_line_words.append(text)
            
            # 페이지 내 마지막 줄 저장
            if current_line_words:
                line_text = " ".join(current_line_words)
                extracted_lines.append((line_text, round(current_font_size, 1), page_num))
    
    return extracted_lines

# 3. 폰트별 라인 수를 기반으로 본문/소제목 폰트 추정
def estimate_fonts(extracted_lines):
    font_counts = {}
    for line, font, page in extracted_lines:
        # 한글이 포함된 줄만 고려 (그림, 표 등은 제외)
        if re.search(r"[가-힣]", line):
            font_counts[font] = font_counts.get(font, 0) + 1
    
    if not font_counts:
        return None, None
    
    # 본문 폰트: 가장 많이 등장한 폰트 크기
    body_font = max(font_counts, key=lambda k: font_counts[k])
    
    # 소제목 폰트: 본문보다 큰 폰트 중 가장 많이 등장한 폰트 크기 (없으면 본문 폰트로 설정)
    candidate_sub = {font: count for font, count in font_counts.items() if font > body_font}
    sub_font = max(candidate_sub, key=lambda k: candidate_sub[k]) if candidate_sub else body_font
    
    return body_font, sub_font

# 4. 추출된 라인에 대해 텍스트 클리닝 및 컨텍스트 타입 지정
def process_extracted_lines(extracted_lines, body_font, sub_font):
    processed_lines = []  # 최종 결과: (텍스트, 폰트크기, 페이지, 컨텍스트 타입)
    
    for line, font, page in extracted_lines:
        # 한글이 포함되지 않는 라인은 제거 (그림 및 표로 간주)
        if not re.search(r"[가-힣]", line):
            continue
        
        # 텍스트 클리닝
        cleaned_line = clean_text(line)
        if not cleaned_line:
            continue
        
        # 폰트 크기가 본문 또는 소제목으로 추정되는 경우에만 저장
        if round(font, 1) == round(body_font, 1):
            context = "본문"
        elif round(font, 1) == round(sub_font, 1):
            context = "소제목"
        else:
            continue  # 본문, 소제목 이외는 제외
        
        processed_lines.append((cleaned_line, font, page, context))
    
    return processed_lines

# 5. 전체 파이프라인 실행 함수
def main(pdf_path):
    # 1) PDF에서 라인별 텍스트, 폰트, 페이지 정보 추출
    extracted_lines = extract_lines_from_pdf(pdf_path)
    
    # 2) 폰트별 라인 수를 확인하여 본문/소제목 폰트 크기 추정
    body_font, sub_font = estimate_fonts(extracted_lines)
    
    # 3) 추출된 변수에서 전처리 및 컨텍스트 타입 지정 (본문/소제목인 경우만 남김)
    processed_lines = process_extracted_lines(extracted_lines, body_font, sub_font)
    
    return processed_lines, body_font, sub_font

# 6. 코드 실행 예시
pdf_path = "../../data/pdf/강관비계 안전작업지침.pdf"  # PDF 파일 경로 설정
processed_lines, body_font, sub_font = main(pdf_path)

print(f"추론된 본문 폰트 크기: {body_font}")
print(f"추론된 소제목 폰트 크기: {sub_font}")
print("샘플 결과 (최대 10개):")
for line in processed_lines[:10]:
    print(line)


추론된 본문 폰트 크기: 12.0
추론된 소제목 폰트 크기: 13.0
샘플 결과 (최대 10개):
('1. 목 적', 13.0, 3, '소제목')
('이 지침은 산업안전보건기준에 관한 규칙이하 안전보건규칙이라 한다 제1편', 12.0, 3, '본문')
('제7장비계의 규정에 의하여 강관비계 안전보건작업 관한 기술적 사항을 정함을', 12.0, 3, '본문')
('목적으로 한다.', 12.0, 3, '본문')
('2. 적용범위', 13.0, 3, '소제목')
('이 지침은 강관비계를 설치 및 사용하는 모든 건설공사에 적용한다.', 12.0, 3, '본문')
('3. 용어의 정의', 13.0, 3, '소제목')
('이 지침에서 사용되는 용어의 정의는 다음과 같다.', 12.0, 3, '본문')
('가 비계라 함은 공사용 통로나 작업발판 설치를 위하여 구조물의 주위에 조립,', 12.0, 3, '본문')
('설치되는 가설구조물을 말한다.', 12.0, 3, '본문')


In [7]:
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer

# SentenceTransformer를 래핑한 커스텀 임베딩 클래스
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> list[float]:
        embedding = self.model.encode(text)
        return embedding.tolist()


# Document 객체 생성 (본문 텍스트만 임베딩, 메타데이터에 소제목과 페이지 번호 포함)
documents = []
current_subtitle = None  # 현재 적용할 소제목(본문에 연결)
for text, font, page, context in processed_lines:
    if context == "소제목":
        # 소제목은 이후 본문에 대한 메타데이터로 활용하기 위해 저장
        current_subtitle = text
    elif context == "본문":
        # 본문 텍스트만 임베딩에 사용, 메타데이터에 소제목과 페이지 번호 저장
        doc = Document(
            page_content=text,
            metadata={"page": page, "소제목": current_subtitle}
        )
        documents.append(doc)

print(f"임베딩할 문서 수: {len(documents)}")

# SentenceTransformer 임베딩 객체 생성 및 Chroma 벡터 DB 저장
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformerEmbeddings(embedding_model_name)

db = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory='db/chroma_db',
    collection_name='construction_manual'
)

print("벡터 DB에 문서가 성공적으로 저장되었습니다.")



임베딩할 문서 수: 230


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

벡터 DB에 문서가 성공적으로 저장되었습니다.


In [8]:
db.get('57c182c5-0401-4c36-b47a-e46a8868e65d')

{'ids': ['57c182c5-0401-4c36-b47a-e46a8868e65d'],
 'embeddings': None,
 'documents': ['이 지침에서 사용되는 용어의 정의는 다음과 같다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'page': 3, '소제목': '3. 용어의 정의'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [9]:
result = db.get(where={"소제목": "3. 용어의 정의"})
print(result)

{'ids': ['57c182c5-0401-4c36-b47a-e46a8868e65d', '531306d5-16a7-4512-ab2d-b5dd721ff407', 'f8fc88b6-1766-4174-81aa-2571f550ffee', 'a45ccf8c-9c61-416a-8375-27b8433ad117', '59026b96-015a-47d1-beac-f42762b5fbe2', 'ba049e33-e884-4dc8-8025-c10c021ff59d', '0c488082-dc5c-428f-9e4e-28d82ddefd5c', '8bf4fce2-0ff9-4061-a582-0f2afdfa4d66', 'f2e5eb46-8d00-4ec4-a4b8-fe2fe1342fb1', 'b3158f0a-fb6b-427c-94f6-c1abb92a3a0e', 'c9130fa5-d26e-4d04-87ef-2bac5ffce2ef', 'cee5772f-c810-43fb-aaed-464a86628fbe', 'e0de812c-be1a-457e-bcc6-b42ac97c2f41', 'f6b2b0f3-790e-450a-99e3-d057fa293a7f', '231f0db4-0866-4184-9724-809ec8180197', '9ba96d59-5e95-42c7-baff-662e2befae22', 'faf8bf89-3591-4a91-a6d1-199ff0d0a844', 'e0265588-8221-49c6-aeed-5a101632f7ea', '186dc125-d38e-433a-9351-22a811233f22', 'a0ddd5e7-642a-4d65-a65f-573286d1d7f7', 'd9cd54e7-3e4e-49e8-b1ad-2394208f5b9b', '08361441-6912-48ed-bf2a-95451d0cd63c', '4c0d3dda-bc82-446f-88e0-de4d968a1101', '00494972-c78a-4a1e-bd9d-5e1199502501', '3a9d552f-1988-49f2-8b68-1566c3